In [12]:
'''

Challenge Scope:
    
1. replicate HTTP request that sources the open positions for TimsCareers
2. the method should take:
    - a location (lat+lng or postal code)
    - a results parameter (example: only 10 results back)
3. it should return the following attributes for each open position:
    - title
    - full time/part time
    - requirements, if any

'''

'\n\nChallenge Scope:\n    \n1. replicate HTTP request that sources the open positions for TimsCareers\n2. the method should take:\n    - a location (lat+lng or postal code)\n    - a results parameter (example: only 10 results back)\n3. it should return the following attributes for each open position:\n    - title\n    - full time/part time\n    - requirements, if any\n\n'

In [13]:
import pgeocode

nomi=pgeocode.Nominatim('ca')

nomi.query_location('union station')

,country_code,postal_code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
692,CA,M5J,Downtown Toronto (Harbourfront East / Union St...,Ontario,ON,Toronto,8133394.0,NaN,NaN,43.623,-79.3936,5.0


In [14]:
import requests
from pgeocode import Nominatim
import pandas as pd


def get_jobs_for(lat=None, lng=None, postal_code=None, results=20):

    if (lat is None or lng is None) and postal_code is None:
        raise ValueError('Both lat and lng must be provided')
    
    if postal_code is not None:
        nomi=Nominatim('ca')
        geo=nomi.query_postal_code(postal_code)
        lat=geo.latitude
        lng=geo.longitude

    url = f"https://api.higherme.com/classic/jobs?page=1&includes=location,location.company,location.externalServiceReferences&limit={results}&filters^[brand.id^]=58bd9e7f472bd&filters^[lat^]={lat}&filters^[lng^]={lng}&filters^[distance^]=6.25&sort^[distance^]=asc"

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/115.0',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'HigherMe-Client-Version': '2023.07.14_20.2',
        'Origin': 'https://app.higherme.com',
        'Connection': 'keep-alive',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-site',
        'TE': 'trailers'
    }

    response = requests.get(url, headers=headers)

    response_data=response.json()

    df = pd.DataFrame(
        data=[r.get('attributes') for r in response_data.get('data')],
        columns=['title', 'full_time', 'part_time', 'requirements']
    )

    return df

In [15]:
get_jobs_for(43.6532,-79.3832,10)

,title,full_time,part_time,requirements
0,Prep Cook,True,True,"[Serve Safe certified, CPR/Choke-saver certifi..."
1,Team Member,True,True,"[Serve Safe certified, CPR/Choke-saver certifi..."
2,Team Lead,True,True,"[Sign-off of all stations, including cook posi..."
3,General Manager,True,False,[Previous restaurant management experience req...
4,Team Member,True,True,"[Serve Safe certified, CPR/Choke-saver certifi..."
5,Delivery Driver,False,True,"[Passion for food, A commitment to providing g..."
6,Line Cook,True,False,"[Passion for food, A commitment to providing g..."
7,Manager,True,False,"[Any experience in the restaurant industry, Mu..."
8,Assistant Manager,False,True,[1+ year of experience working in any role in ...
9,CiCi's Pizza Team Member,True,True,"[Must be enthusiastic with a great attitude, A..."


In [16]:
get_jobs_for(postal_code='m5e',results=10)

,title,full_time,part_time,requirements
0,Prep Cook,True,True,"[Serve Safe certified, CPR/Choke-saver certifi..."
1,Team Member,True,True,"[Serve Safe certified, CPR/Choke-saver certifi..."
2,Team Lead,True,True,"[Sign-off of all stations, including cook posi..."
3,General Manager,True,False,[Previous restaurant management experience req...
4,Team Member,True,True,"[Serve Safe certified, CPR/Choke-saver certifi..."
5,Delivery Driver,False,True,"[Passion for food, A commitment to providing g..."
6,Line Cook,True,False,"[Passion for food, A commitment to providing g..."
7,Manager,True,False,"[Any experience in the restaurant industry, Mu..."
8,Assistant Manager,False,True,[1+ year of experience working in any role in ...
9,CiCi's Pizza Team Member,True,True,"[Must be enthusiastic with a great attitude, A..."


In [17]:
# Export
# df.to_csv('open_positions.csv',index=False)